## Ranking Analysis
===============================================================================
Purpose:
    - To rank items (e.g., products, customers) based on performance or other metrics.
    - To identify top performers or laggards.

In [0]:
--Top 5 best selling products
SELECT *
FROM (
    SELECT
        p.ProductName,
        round(SUM(of.Quantity * of.UnitPrice),2) AS total_revenue,
        RANK() OVER (ORDER BY SUM(of.Quantity * of.UnitPrice) DESC) AS rank_products
    FROM gold.orders_fact of
    LEFT JOIN gold.products p
        ON p.ProductID = of.ProductID
    where p.active_flg = 1
    GROUP BY p.ProductName
) AS ranked_products
WHERE rank_products <= 5;

## Change Over Time Analysis
===============================================================================
Purpose:
    - To track trends, growth, and changes in key metrics over time.
    - For time-series analysis and identifying seasonality.
    - To measure growth or decline over specific periods.

In [0]:
SELECT
    date_trunc('month', LastEditedWhen) AS order_date,
    round(SUM(Quantity * UnitPrice),2) AS total_revenue,
    COUNT(DISTINCT CustomerID) AS total_customers,
    SUM(quantity) AS total_quantity
FROM gold.orders_fact
WHERE LastEditedWhen IS NOT NULL and active_flg = 1
GROUP BY date_trunc('month', LastEditedWhen)
ORDER BY date_trunc('month', LastEditedWhen);

## Cumulative Analysis
===============================================================================
Purpose:
    - To calculate running totals or moving averages for key metrics.
    - To track performance over time cumulatively.
    - Useful for growth analysis or identifying long-term trends.

In [0]:
-- Calculate the total sales per month 
-- and the running total of sales over time 
SELECT
	order_year,
	order_month,
	SUM(total_revenue) OVER (ORDER BY order_year, order_month) AS running_total_revenue,  
	AVG(avg_price) OVER (ORDER BY order_year, order_month) AS moving_average_price
FROM
(
    SELECT 
        date_trunc('year', LastEditedWhen) AS order_year,
        date_trunc('month', LastEditedWhen) AS order_month,
        SUM(Quantity * UnitPrice) AS total_revenue,
        AVG(UnitPrice) AS avg_price
    FROM gold.orders_fact
    WHERE LastEditedWhen IS NOT NULL and active_flg = 1
    GROUP BY date_trunc('year', LastEditedWhen),
          date_trunc('month', LastEditedWhen)
) t


## Performance Analysis (Year-over-Year, Month-over-Month)
===============================================================================
Purpose:
    - To measure the performance of products, customers, or regions over time.
    - For benchmarking and identifying high-performing entities.
    - To track yearly trends and growth.

In [0]:
/* Analyze the yearly performance of products by comparing their revenue 
to both the average revenue performance of the product and the previous year's revenue */
WITH yearly_product_revenue AS (
    SELECT
        YEAR(of.LastEditedWhen) AS order_year,
        p.ProductName,
        round(SUM(of.Quantity * of.UnitPrice),2) AS current_revenue
    FROM gold.orders_fact of
    LEFT JOIN gold.products p
        ON of.ProductID = p.ProductID
    WHERE of.LastEditedWhen IS NOT NULL and p.active_flg = 1
    GROUP BY 
        YEAR(of.LastEditedWhen),
        p.ProductName
)
SELECT
    order_year,
    ProductName,
    current_revenue,
    AVG(current_revenue) OVER (PARTITION BY ProductName) AS avg_revenue,
    current_revenue - AVG(current_revenue) OVER (PARTITION BY ProductName) AS diff_avg,
    CASE 
        WHEN current_revenue - AVG(current_revenue) OVER (PARTITION BY ProductName) > 0 THEN 'Above Avg'
        WHEN current_revenue - AVG(current_revenue) OVER (PARTITION BY ProductName) < 0 THEN 'Below Avg'
        ELSE 'Avg'
    END AS avg_change,
    -- Year-over-Year Analysis
    LAG(current_revenue) OVER (PARTITION BY ProductName ORDER BY order_year) AS py_revenue,
    current_revenue - LAG(current_revenue) OVER (PARTITION BY ProductName ORDER BY order_year) AS diff_py,
    CASE 
        WHEN current_revenue - LAG(current_revenue) OVER (PARTITION BY ProductName ORDER BY order_year) > 0 THEN 'Increase'
        WHEN current_revenue - LAG(current_revenue) OVER (PARTITION BY ProductName ORDER BY order_year) < 0 THEN 'Decrease'
        ELSE 'No Change'
    END AS py_change
FROM yearly_product_revenue
ORDER BY ProductName, order_year;

## Part-to-Whole Analysis
===============================================================================
Purpose:
    - To compare performance or metrics across dimensions or time periods.
    - To evaluate differences between categories.
    - Useful for A/B testing or regional comparisons.

In [0]:
-- Which products contribute the most to overall revenue?
WITH product_revenue AS (
    SELECT
        p.ProductName,
        round(SUM(of.Quantity * of.UnitPrice),2) AS total_revenue
    FROM gold.orders_fact of
    LEFT JOIN gold.products p
        ON p.ProductID = of.ProductID
    GROUP BY p.ProductName
)
SELECT
    ProductName,
    total_revenue,
    SUM(total_revenue) OVER () AS overall_revenue,
    ROUND((CAST(total_revenue AS FLOAT) / SUM(total_revenue) OVER ()) * 100, 2) AS percentage_of_total
FROM product_revenue
ORDER BY total_revenue DESC;